In [1]:
import numpy as np
import pandas as pd
from sklearn import naive_bayes

In [2]:
from collections import defaultdict

In [3]:
#can replace the first two with the third but keeping all for now
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split, StratifiedShuffleSplit 

In [5]:
from sklearn.pipeline import Pipeline

In [6]:
from sklearn.grid_search import GridSearchCV

In [7]:
from sklearn.feature_selection import SelectKBest, chi2

In [8]:
pd.set_option('display.max_columns', 50)

In [9]:
#first get the data in shape
#raw_data = pd.read_csv("Execution-Defect_Report_csv.csv", header=7)
raw_data = pd.read_csv("Execution-Defect_Report_csv_fixed.csv", header=7) #fixed has cleaned comments
raw_data.head(3)

,Defect ID,Vendor Defect ID,Summary,Description,Status,Severity,Detected on Date,Unnamed: 7,Defect Age(# of buisness days open),Detected By,Assigned To,Actual Fix Time,Estimated Fix Date,Modified,Priority,Project,Reproducible,Comments_orig,Vendor Name,Root Cause,Comments
0,132,0,Appl 50000446 Rates calculated incorrectly,Test Set: Credit_Pricing_And_Tiers\rTest: CLOS...,Closed,3-High,9/9/13,NaN,NaN,15306,15306,2,NaN,9/13/13 8:39,1-Low,NaN,Y,"Salvatore Mule <015306>, 9/9/2013: Spoke to Je...",NaN,Tester error,"Salvatore Mule <015306>, 9/9/2013: Spoke to Je..."
1,172,101,app 50000132 first app of the day locked from ...,app 50000132 first app of the day locked from ...,Closed,1-Low,8/5/13,NaN,NaN,15424,19931,60,12/25/13,10/3/13 15:03,2-Medium,NaN,N,"Tom Sokolski <015424>, 10/3/2013: closed same ...",CMSI,Duplicate,"Tom Sokolski <015424>, 10/3/2013: closed same ..."
2,102,102 or 158 now 158,app 50000106 dealer reserve was showing $1000 ...,app 50000106 dealer reserve was showing $1000 ...,Closed,2-Medium,8/29/13,NaN,NaN,15424,15424,23,9/20/13,9/20/13 15:57,3-High,NaN,Y,"Tamara Osifchin <018494>, 8/29/2013: assigning...",CMSI,Code defect,"Tamara Osifchin <018494>, 8/29/2013: assigning..."


In [10]:
raw_data.rename(columns=lambda x: x.replace(" ","_"), inplace=True)
raw_data.head(3)

,Defect_ID,Vendor_Defect_ID,Summary,Description,Status,Severity,Detected_on_Date,Unnamed:_7,Defect_Age(#_of_buisness_days_open),Detected_By,Assigned_To,Actual_Fix_Time,Estimated_Fix_Date,Modified,Priority,Project,Reproducible,Comments_orig,Vendor_Name,Root_Cause,Comments
0,132,0,Appl 50000446 Rates calculated incorrectly,Test Set: Credit_Pricing_And_Tiers\rTest: CLOS...,Closed,3-High,9/9/13,NaN,NaN,15306,15306,2,NaN,9/13/13 8:39,1-Low,NaN,Y,"Salvatore Mule <015306>, 9/9/2013: Spoke to Je...",NaN,Tester error,"Salvatore Mule <015306>, 9/9/2013: Spoke to Je..."
1,172,101,app 50000132 first app of the day locked from ...,app 50000132 first app of the day locked from ...,Closed,1-Low,8/5/13,NaN,NaN,15424,19931,60,12/25/13,10/3/13 15:03,2-Medium,NaN,N,"Tom Sokolski <015424>, 10/3/2013: closed same ...",CMSI,Duplicate,"Tom Sokolski <015424>, 10/3/2013: closed same ..."
2,102,102 or 158 now 158,app 50000106 dealer reserve was showing $1000 ...,app 50000106 dealer reserve was showing $1000 ...,Closed,2-Medium,8/29/13,NaN,NaN,15424,15424,23,9/20/13,9/20/13 15:57,3-High,NaN,Y,"Tamara Osifchin <018494>, 8/29/2013: assigning...",CMSI,Code defect,"Tamara Osifchin <018494>, 8/29/2013: assigning..."


In [11]:
#but for now this seems to work:
raw_data.Comments = raw_data.Comments.str.decode('utf-8','ignore') 

In [12]:
tmp = raw_data[raw_data.Comments.isnull()]
tmp

,Defect_ID,Vendor_Defect_ID,Summary,Description,Status,Severity,Detected_on_Date,Unnamed:_7,Defect_Age(#_of_buisness_days_open),Detected_By,Assigned_To,Actual_Fix_Time,Estimated_Fix_Date,Modified,Priority,Project,Reproducible,Comments_orig,Vendor_Name,Root_Cause,Comments
293,454,NaN,No Longer have the ability to un-book loans,"Under my profile, FNBISKRAJEWSKI I no longer h...",New,3-High,12/5/13,12/5/13,1,15425,NaN,NaN,NaN,12/5/13 9:40,NaN,NaN,Y,NaN,NaN,TBD,NaN


In [13]:
raw_data.Comments = raw_data.Comments.fillna("")
tmp = raw_data[raw_data.Comments.isnull()]
tmp

,Defect_ID,Vendor_Defect_ID,Summary,Description,Status,Severity,Detected_on_Date,Unnamed:_7,Defect_Age(#_of_buisness_days_open),Detected_By,Assigned_To,Actual_Fix_Time,Estimated_Fix_Date,Modified,Priority,Project,Reproducible,Comments_orig,Vendor_Name,Root_Cause,Comments


In [13]:
raw_data.dtypes

Defect_ID                                int64
Vendor_Defect_ID                        object
Summary                                 object
Description                             object
Status                                  object
Severity                                object
Detected_on_Date                        object
Unnamed:_7                              object
Defect_Age(#_of_buisness_days_open)    float64
Detected_By                              int64
Assigned_To                            float64
Actual_Fix_Time                        float64
Estimated_Fix_Date                      object
Modified                                object
Priority                                object
Project                                float64
Reproducible                            object
Comments_orig                           object
Vendor_Name                             object
Root_Cause                              object
Comments                                object
dtype: object

In [14]:
raw_data[raw_data.Root_Cause.isnull()].head(3)

,Defect_ID,Vendor_Defect_ID,Summary,Description,Status,Severity,Detected_on_Date,Unnamed:_7,Defect_Age(#_of_buisness_days_open),Detected_By,Assigned_To,Actual_Fix_Time,Estimated_Fix_Date,Modified,Priority,Project,Reproducible,Comments_orig,Vendor_Name,Root_Cause,Comments
309,349,395D1206,App# 50002011 Did not reflect proper payment a...,Test Set: Credit_Credit_Bureaus\rTest: CLOS_ C...,Open,2-Medium,11/15/13,12/5/13,15,15306,19931,NaN,NaN,11/27/13 11:09,3-High,NaN,Y,"Tamara Osifchin <018494>, 11/15/2013: Payment ...",NaN,NaN,"Tamara Osifchin <018494>, 11/15/2013: Payment ..."
310,362,407,App# 50002129 should only have 1 rule for max ...,App# 50002129 has 2 review rules for max mark-...,Open,2-Medium,11/20/13,12/5/13,12,15306,18000,NaN,NaN,12/3/13 12:12,2-Medium,NaN,Y,"Tamara Osifchin <018494>, 11/20/2013: medium p...",NaN,NaN,"Tamara Osifchin <018494>, 11/20/2013: medium p..."
311,365,409Dnextbuild,Region Code on Originator Aging Statistics is ...,The Region Code on the screen labeled as Origi...,Open,2-Medium,11/21/13,12/5/13,11,11798,19931,NaN,NaN,12/2/13 16:29,1-Low,NaN,Y,"Tamara Osifchin <018494>, 11/21/2013: low prio...",NaN,NaN,"Tamara Osifchin <018494>, 11/21/2013: low prio..."


In [15]:
raw_data.Root_Cause.value_counts()

Code defect          213
Requirement issue     30
Tester error          29
Not reproducible      23
Duplicate              8
TBD                    6
dtype: int64

In [16]:
raw_data[raw_data.Root_Cause == 'TBD']

,Defect_ID,Vendor_Defect_ID,Summary,Description,Status,Severity,Detected_on_Date,Unnamed:_7,Defect_Age(#_of_buisness_days_open),Detected_By,Assigned_To,Actual_Fix_Time,Estimated_Fix_Date,Modified,Priority,Project,Reproducible,Comments_orig,Vendor_Name,Root_Cause,Comments
67,281,328D,Collateral Section for funding needs to look a...,Collateral section under Contract Infoneeds to...,Reopen,4-Very High,10/23/13,12/5/13,32,15425,19931,8,NaN,12/2/13 16:17,4-Very High,NaN,Y,"Tamara Osifchin <018494>, 10/23/2013: Jeddalyn...",NaN,TBD,"Tamara Osifchin <018494>, 10/23/2013: Jeddalyn..."
217,427,417,App# 50002195 Buy rate did not recalculate on ...,App# 50002195 has incorrect buy rate when amou...,Open,4-Very High,11/26/13,12/5/13,8,15306,18000,NaN,NaN,12/3/13 14:23,3-High,NaN,Y,"Tamara Osifchin <018494>, 11/26/2013: Similar ...",NaN,TBD,"Tamara Osifchin <018494>, 11/26/2013: Similar ..."
250,357,401,"VINTEK is not operational at this time, unable...",Test Set: Vintek\rTest: CLOS_ADMIN__Vintek_Bat...,Open,3-High,11/19/13,12/5/13,13,5778,19530,NaN,NaN,12/2/13 16:23,3-High,NaN,Y,Failed this test since VINTEK is not functiona...,NaN,TBD,Failed this test since VINTEK is not functiona...
253,360,404,NACHA file is not operational at this time to ...,Test Set: ACH\rTest: CLOS_ADMIN___ACH_001_NACH...,Open,3-High,11/19/13,12/5/13,13,5778,19530,NaN,NaN,12/2/13 16:23,3-High,NaN,Y,"David Pepe <005778>, 11/19/2013: Failed this s...",NaN,TBD,"David Pepe <005778>, 11/19/2013: Failed this s..."
287,441,422,Dealer Reserve screen is not updating after ne...,"app 50002213 completed contract info with 10,0...",Open,3-High,11/29/13,12/5/13,5,15424,19931,NaN,NaN,12/4/13 16:13,3-High,NaN,Y,"Tamara Osifchin <018494>, 12/2/2013: Tom logge...",NaN,TBD,"Tamara Osifchin <018494>, 12/2/2013: Tom logge..."
293,454,NaN,No Longer have the ability to un-book loans,"Under my profile, FNBISKRAJEWSKI I no longer h...",New,3-High,12/5/13,12/5/13,1,15425,NaN,NaN,NaN,12/5/13 9:40,NaN,NaN,Y,NaN,NaN,TBD,


In [17]:
raw_data = raw_data[raw_data.Root_Cause != 'TBD']
print raw_data.shape
print "number of rows with root cause value: ", sum(raw_data.Root_Cause.value_counts())

(373, 21)
number of rows with root cause value:  303


In [18]:
#need to create the vocabulary from all the data (i.e. both train and test splits) 
#but will keep it at 300 for now to make the math easier
print raw_data.shape
raw_data = raw_data[:300]
raw_data.shape

(373, 21)


(300, 21)

In [19]:
raw_data[raw_data.Root_Cause.isnull()].head(3)

,Defect_ID,Vendor_Defect_ID,Summary,Description,Status,Severity,Detected_on_Date,Unnamed:_7,Defect_Age(#_of_buisness_days_open),Detected_By,Assigned_To,Actual_Fix_Time,Estimated_Fix_Date,Modified,Priority,Project,Reproducible,Comments_orig,Vendor_Name,Root_Cause,Comments


In [20]:
raw_data.iloc[:3,:5]

,Defect_ID,Vendor_Defect_ID,Summary,Description,Status
0,132,0,Appl 50000446 Rates calculated incorrectly,Test Set: Credit_Pricing_And_Tiers\rTest: CLOS...,Closed
1,172,101,app 50000132 first app of the day locked from ...,app 50000132 first app of the day locked from ...,Closed
2,102,102 or 158 now 158,app 50000106 dealer reserve was showing $1000 ...,app 50000106 dealer reserve was showing $1000 ...,Closed


In [21]:
raw_data.sort('Defect_ID',inplace=True)
raw_data.iloc[:,:6].head(3)
raw_data.head(3)

,Defect_ID,Vendor_Defect_ID,Summary,Description,Status,Severity,Detected_on_Date,Unnamed:_7,Defect_Age(#_of_buisness_days_open),Detected_By,Assigned_To,Actual_Fix_Time,Estimated_Fix_Date,Modified,Priority,Project,Reproducible,Comments_orig,Vendor_Name,Root_Cause,Comments
155,2,27,auto decline does not populate decline reasons...,Application 9893 - no reasons printedon the ad...,Closed,2-Medium,5/31/13,NaN,NaN,18494,15306,27,NaN,9/13/13 17:04,3-High,NaN,Y,"Tamara Osifchin <018494>, 6/27/2013: Per Laure...",CMSI,Requirement issue,"Tamara Osifchin <018494>, 6/27/2013: Per Laure..."
231,8,34,rates should be coded per model year,Rates should be coded per model yeareven thoug...,Closed,2-Medium,6/4/13,NaN,NaN,18494,5778,34,NaN,7/26/13 13:38,NaN,NaN,Y,"Tamara Osifchin <018494>, 6/27/2013: Fix is in...",CMSI,Code defect,"Tamara Osifchin <018494>, 6/27/2013: Fix is in..."
291,11,40,Remove tolerance rules rows 13 and 14,"Per discussion at 6/5 tolerance rulesmeeting, ...",Closed,2-Medium,6/5/13,NaN,NaN,18494,11798,40,NaN,8/8/13 9:41,3-High,NaN,Y,"Tamara Osifchin <018494>, 6/27/2013: Per Laure...",CMSI,Code defect,"Tamara Osifchin <018494>, 6/27/2013: Per Laure..."


In [22]:
raw_data.reset_index(drop=True, inplace=True)
raw_data.iloc[:3,:5]

,Defect_ID,Vendor_Defect_ID,Summary,Description,Status
0,2,27,auto decline does not populate decline reasons...,Application 9893 - no reasons printedon the ad...,Closed
1,8,34,rates should be coded per model year,Rates should be coded per model yeareven thoug...,Closed
2,11,40,Remove tolerance rules rows 13 and 14,"Per discussion at 6/5 tolerance rulesmeeting, ...",Closed


In [23]:
raw_data.Root_Cause.value_counts()

Code defect          211
Requirement issue     29
Tester error          29
Not reproducible      23
Duplicate              8
dtype: int64

In [24]:
#null accuracy rate?  whatever this is called -- it is the rate to beat
raw_data.Root_Cause.value_counts().max()/float(len(raw_data.Root_Cause))

0.7033333333333334

In [25]:
defects = defaultdict(list)

In [99]:
raw_data.columns

Index([u'Defect_ID', u'Vendor_Defect_ID', u'Summary', u'Description',
       u'Status', u'Severity', u'Detected_on_Date', u'Unnamed:_7',
       u'Defect_Age(#_of_buisness_days_open)', u'Detected_By', u'Assigned_To',
       u'Actual_Fix_Time', u'Estimated_Fix_Date', u'Modified', u'Priority',
       u'Project', u'Reproducible', u'Comments', u'Vendor_Name',
       u'Root_Cause'],
      dtype='object')

In [26]:
for i in range(len(raw_data.Comments)):
    defects[raw_data.Defect_ID.iloc[i]].append(raw_data.Comments.iloc[i])

In [28]:
type(raw_data.Root_Cause.iloc[1])

str

In [31]:
for defect_id, comment in defects.iteritems():
    print "def: ", defect_id, "comment: ", comment, "\n"

def:  2 comment:  [u'Tamara Osifchin <018494>, 6/27/2013: Per Lauren FNFG needs to provide requirements.  Compliance needs to look at the mapping. Will probably want to populate a reason for auto declines.\r\r________________________________________\rTamara Osifchin <018494>, 6/27/2013: assigned to Lisa K but Dave C is the real assignee\r\r________________________________________\rTamara Osifchin <018494>, 7/18/2013: Lauren is waiting on FNFG approval of the decline reasons mapping\r\r________________________________________\rTamara Osifchin <018494>, 9/12/2013: Assigning to Sam to verify that some decline reasons are displayed  on adverse action letter  on system decline now.  Will create a new question/change request for the decline reasons mapping.\r\r________________________________________\rSalvatore Mule <015306>, 9/13/2013: Closed defect: application #50000530 and adverse action reasons appear on auto declines.'] 

def:  8 comment:  [u'Tamara Osifchin <018494>, 6/27/2013: Fix is

In [27]:
#convert the comment text from a list of chars to a string
for defect_id, comment_text in defects.iteritems():
    defects[defect_id] = "".join(comment_text)
    
#corpus will contain only the comment text of each defect
corpus = []
for id, defect in sorted(defects.iteritems(), key=lambda t: int(t[0])):
    corpus.append(defect)
    


In [28]:
type(corpus)

list

In [34]:
print len(corpus)

300


In [102]:
#could create the count vector and transform it to idfs in one step with TfidfVectorizer but I wanted
#to do it in 2 steps first
cvec = CountVectorizer(ngram_range=(1,2),stop_words = 'english')

In [142]:
#what happens if I give it a vocabulary
vocab = {'duplicate':0,'change request':1,'bucket':2,
         'fix description':3,'retry':4,'back':5,'dev':6,
         'in progress':7, 'duplicate of':8, 
         'requirement':9,'new requirement':10,'new requirements':11,
         'tester confused':12, 'same as':13,'monitoring':14,
         'rforct':15, 'port to tb':16, 'set to fixed':17, 'in a while':18, 'have not seen':19,
         'fix desc':20, 'requirements':21, 'could not reproduce':22, 'intermittently':23,'intermittent':24,
         'script was incorrect':25, 'ready for client testing':26}
cvec = CountVectorizer(ngram_range=(1,4),vocabulary=vocab)

In [143]:
cvec.fit(corpus)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 4), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None,
        vocabulary={'in progress': 7, 'requirement': 9, 'back': 5, 'fix description': 3, 'ready for client testing': 26, 'rforct': 15, 'new requirement': 10, 'retry': 4, 'monitoring': 14, 'script was incorrect': 25, 'change request': 1, 'new requirements': 11, 'duplicate': 0, 'have not seen': 19, 'in a whil... 'intermittently': 23, 'set to fixed': 17, 'tester confused': 12, 'fix desc': 20, 'duplicate of': 8})

In [37]:
cvec.vocabulary_

{'back': 5,
 'bucket': 2,
 'change request': 1,
 'could not reproduce': 22,
 'dev': 6,
 'duplicate': 0,
 'duplicate of': 8,
 'fix desc': 20,
 'fix description': 3,
 'have not seen': 19,
 'in a while': 18,
 'in progress': 7,
 'intermittent': 24,
 'intermittently': 23,
 'monitoring': 14,
 'new requirement': 10,
 'new requirements': 11,
 'port to tb': 16,
 'ready for client testing': 26,
 'requirement': 9,
 'requirements': 21,
 'retry': 4,
 'rforct': 15,
 'same as': 13,
 'script was incorrect': 25,
 'set to fixed': 17,
 'tester confused': 12}

In [47]:
cvec

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None,
        vocabulary={'in progress': 7, 'requirement': 9, 'back': 5, 'fix description': 3, 'ready for client testing': 26, 'rforct': 15, 'new requirement': 10, 'retry': 4, 'monitoring': 14, 'script was incorrect': 25, 'change request': 1, 'new requirements': 11, 'duplicate': 0, 'have not seen': 19, 'in a whil... 'intermittently': 23, 'set to fixed': 17, 'tester confused': 12, 'fix desc': 20, 'duplicate of': 8})

In [144]:
#create the sparse matrix of term counts
cmatrix = cvec.transform(corpus)
cmatrix

<300x27 sparse matrix of type '<type 'numpy.int64'>'
	with 375 stored elements in Compressed Sparse Row format>

In [105]:
#create the sparse matrix of term counts with pre-cleaned data
cmatrix = cvec.transform(corpus)
cmatrix

<300x27 sparse matrix of type '<type 'numpy.int64'>'
	with 186 stored elements in Compressed Sparse Row format>

In [106]:
#this was before I cleaned the text of the comments: 358, after 385
print cmatrix.nnz #non-zeros

186


In [145]:
#this is after I cleaned the comments by splitting merged words
print cmatrix.nnz

375


In [113]:
cmatrix.todense().shape

(300, 27)

In [104]:
idx = pd.Series(cvec.vocabulary_)
b = idx.order().index
term_counts = pd.DataFrame(cmatrix.todense(),columns=b.values)
term_counts.rename(columns=lambda x: x.replace(" ","_"),inplace=True)
term_counts.sum(axis=0)

duplicate                   20
change_request              32
bucket                       9
fix_description             20
retry                        2
back                        83
dev                         44
in_progress                 10
duplicate_of                12
requirement                  2
new_requirement              0
new_requirements             1
tester_confused              1
same_as                      8
monitoring                   8
rforct                       8
port_to_tb                   0
set_to_fixed                 0
in_a_while                   0
have_not_seen                0
fix_desc                     0
requirements                 6
could_not_reproduce          0
intermittently               1
intermittent                 0
script_was_incorrect         0
ready_for_client_testing     0
dtype: int64

In [148]:
#after I fixed the ngram range bug:
idx = pd.Series(cvec.vocabulary_)
b = idx.order().index
term_counts = pd.DataFrame(cmatrix.todense(),columns=b.values)
term_counts.rename(columns=lambda x: x.replace(" ","_"),inplace=True)
term_counts.sum(axis=0)
term_counts[term_counts['ready_for_client_testing'] > 0]

,duplicate,change_request,bucket,fix_description,retry,back,dev,in_progress,duplicate_of,requirement,new_requirement,new_requirements,tester_confused,same_as,monitoring,rforct,port_to_tb,set_to_fixed,in_a_while,have_not_seen,fix_desc,requirements,could_not_reproduce,intermittently,intermittent,script_was_incorrect,ready_for_client_testing
37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
45,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
56,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
83,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
154,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
209,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
214,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
221,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [167]:
text_found = raw_data[['Defect_ID','Comments']].loc[raw_data.Comments.map(lambda x: x.lower()).str.contains('ready for client testing') == True]
text_found
#print raw_data[raw_data.Comments.str.contains("tester confused") == True].shape

,Defect_ID,Comments
37,90,"Bradley Goldowsky <015304>, 8/28/2013: Per Lau..."
45,98,"Tamara Osifchin <018494>, 8/29/2013: Mary was ..."
48,101,"Tamara Osifchin <018494>, 8/29/2013: assigned ..."
56,109,"Bradley Goldowsky <015304>, 9/3/2013: Estimate..."
68,122,"Tamara Osifchin <018494>, 9/11/2013: Moving to..."
83,137,"Tamara Osifchin <018494>, 9/10/2013:updated wi..."
154,212,"Tom Sokolski <015424>, 10/18/2013: cmsi will c..."
209,270,Was able to login on 10/11/13 after changing s...
214,276,"Tamara Osifchin <018494>,10/21/2013: Stacy che..."
221,284,"Tamara Osifchin <018494>, 10/28/2013: Assignin..."


In [159]:
raw_data[['Defect_ID','Comments']].iloc[37]

Defect_ID                                                   90
Comments     Bradley Goldowsky <015304>, 8/28/2013: Per Lau...
Name: 37, dtype: object

In [136]:
print text_found.shape
count_gt_0 = term_counts[term_counts['change_request'] > 0]
print count_gt_0.shape

(24, 2)
(24, 27)


In [115]:
idf_transformer = TfidfTransformer()

In [116]:
idf_transformer.fit(cmatrix)
idf_matrix = idf_transformer.transform(cmatrix)
idf_matrix

<300x27 sparse matrix of type '<type 'numpy.float64'>'
	with 364 stored elements in Compressed Sparse Row format>

In [152]:
#start training the model

In [117]:
y = raw_data.Root_Cause

In [118]:
y.value_counts()

Code defect          211
Requirement issue     29
Tester error          29
Not reproducible      23
Duplicate              8
dtype: int64

In [41]:
print y.iloc[62]
print raw_data.Root_Cause.iloc[62]

Not reproducible
Not reproducible


In [37]:
#use the term freq matrix with stratified train test split, k=1
sss = StratifiedShuffleSplit(y, 1, test_size=0.2, random_state=0)
print len(sss)
#print(sss)       
for train_index, test_index in sss:
    print("TRAIN:", train_index, "TEST:", test_index)
    defect_train, defect_test = cmatrix[train_index], cmatrix[test_index]
    label_train, label_test = y[train_index], y[test_index]

1
('TRAIN:', array([175,  45, 236, 267, 138, 266, 195, 241,  76, 231, 101, 111,  66,
       136, 230, 161, 298, 276,  20, 184, 227,  31, 170,  37,  23,  19,
       164,  16,  64, 157,  21,  95,  90,  86,  67, 265,  44, 229,  82,
       173, 192,  30, 280,  70, 131, 151,  51, 285, 130,  74, 159,  48,
         2,  89,  28, 296, 244, 200, 189, 260, 123, 152,  72, 205, 185,
       144, 177, 281,  87, 206,  26, 274, 194,  85,  18, 249, 262,  99,
       143, 113, 247, 233, 181, 183, 202, 191, 248,   6, 209, 182, 180,
       118, 213, 198,  15,  34, 228,  27, 211,   9, 196, 204, 154, 271,
       193, 208,  10, 108, 216, 272, 187,  54,  35, 109, 245, 254, 282,
        24, 240, 197,  75,  47,  96, 201,  25,  14,  65,   3,  36, 268,
       294, 239, 132, 284,   4,  73, 290, 270,  22, 186,  77, 217, 153,
        60,   7, 199, 259,   0,  97, 141, 232,  78, 137, 105, 162, 158,
       171, 103, 120,  40, 165, 219,  50, 226, 150,  58, 115, 218,   1,
       235, 148, 121, 129, 163,  56,  32, 215, 225,

In [38]:
#use the idf matrix with stratified train test split
sss = StratifiedShuffleSplit(y, 1, test_size=0.2, random_state=0)
print len(sss)
#print(sss)       
for train_index, test_index in sss:
    print("TRAIN:", train_index, "TEST:", test_index)
    idf_defect_train, idf_defect_test = idf_matrix[train_index], idf_matrix[test_index]
    idf_label_train, idf_label_test = y[train_index], y[test_index]


1
('TRAIN:', array([175,  45, 236, 267, 138, 266, 195, 241,  76, 231, 101, 111,  66,
       136, 230, 161, 298, 276,  20, 184, 227,  31, 170,  37,  23,  19,
       164,  16,  64, 157,  21,  95,  90,  86,  67, 265,  44, 229,  82,
       173, 192,  30, 280,  70, 131, 151,  51, 285, 130,  74, 159,  48,
         2,  89,  28, 296, 244, 200, 189, 260, 123, 152,  72, 205, 185,
       144, 177, 281,  87, 206,  26, 274, 194,  85,  18, 249, 262,  99,
       143, 113, 247, 233, 181, 183, 202, 191, 248,   6, 209, 182, 180,
       118, 213, 198,  15,  34, 228,  27, 211,   9, 196, 204, 154, 271,
       193, 208,  10, 108, 216, 272, 187,  54,  35, 109, 245, 254, 282,
        24, 240, 197,  75,  47,  96, 201,  25,  14,  65,   3,  36, 268,
       294, 239, 132, 284,   4,  73, 290, 270,  22, 186,  77, 217, 153,
        60,   7, 199, 259,   0,  97, 141, 232,  78, 137, 105, 162, 158,
       171, 103, 120,  40, 165, 219,  50, 226, 150,  58, 115, 218,   1,
       235, 148, 121, 129, 163,  56,  32, 215, 225,

In [66]:
print idf_defect_train.shape
idf_label_train.size

(243, 27)


pandas.core.series.Series

In [69]:
print idf_defect_test.shape
idf_label_test.size

(57, 27)


57

In [39]:
classifier = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False)

In [51]:
classifier.fit(defect_train,label_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False)

In [117]:
#accuracy score on training set, stratified split, pre-cleaned data
classifier.score(defect_train,label_train)

0.68619246861924688

In [42]:
#accuracy score on training set using stratified split and cleaned data
classifier.score(defect_train,label_train)

0.69874476987447698

In [53]:
test_predictions = classifier.predict(defect_test)
#print test_predictions

In [119]:
#this gives accuracy on the test set, stratified split, pre-cleaned
classifier.score(defect_test,label_test)

0.70491803278688525

In [44]:
#this gives accuracy on the test set using stratified split, cleaned data
classifier.score(defect_test,label_test)

0.77049180327868849

In [54]:
#this gives accuracy on the test set using stratified K fold and cleaned data
classifier.score(defect_test,label_test)

0.78947368421052633

In [56]:
classifier.fit(idf_defect_train,idf_label_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False)

In [121]:
#this gives accuracy on the training set with stratified split and pre-cleaned data
classifier.score(idf_defect_train,idf_label_train)

0.7364016736401674

In [46]:
#this gives accuracy on the training set with stratified split and cleaned data
classifier.score(idf_defect_train,idf_label_train)

0.7489539748953975

In [59]:
idf_test_predictions = classifier.predict(idf_defect_test)
#print idf_test_predictions

In [63]:
#test_probs = classifier.predict_proba(defect_test)
#print np.around(test_probs,decimals=2)

In [123]:
#this gives accuracy on the test set, pre-cleaned data
classifier.score(idf_defect_test,idf_label_test)

0.70491803278688525

In [48]:
#this gives accuracy on the test set with stratified split and cleaned data
classifier.score(idf_defect_test,idf_label_test)

0.77049180327868849

In [75]:
#term frequency matrix and k-fold cross validation
scores = cross_val_score(classifier,  # estimator
                         cmatrix,  # training data
                         y,  # training labels
                         cv=5,  # split data randomly into 10 parts: 9 for training, 1 for scoring
                         scoring='accuracy'  # which scoring metric?
                         )
print scores
print scores.mean()

[ 0.61290323  0.67213115  0.67213115  0.69491525  0.78947368]
0.688310891867


In [168]:
#term frequency matrix and k-fold cross validation after ngram range fix
scores = cross_val_score(classifier,  # estimator
                         cmatrix,  # training data
                         y,  # training labels
                         cv=5,  # split data randomly into 10 parts: 9 for training, 1 for scoring
                         scoring='accuracy'  # which scoring metric?
                         )
print scores
print scores.mean()

[ 0.66129032  0.70491803  0.75409836  0.76271186  0.84210526]
0.745024768718


In [169]:
#idf matrix and k-fold cross validation nope erased that this is now k-fold
scores = cross_val_score(classifier,  # estimator
                         idf_matrix,  # training data
                         y,  # training labels
                         cv=5,  # split data into 5 parts using stratifiedKfold
                         scoring='accuracy'  # which scoring metric?
                         )
print scores
print scores.mean()

[ 0.67741935  0.70491803  0.73770492  0.74576271  0.70175439]
0.713511880698


In [120]:
#idf matrix and k-fold cross validation after ngram range fix
scores = cross_val_score(classifier,  # estimator
                         idf_matrix,  # training data
                         y,  # training labels
                         cv=5,  # split data into 5 parts using stratifiedKfold
                         scoring='accuracy'  # which scoring metric?
                         )
print scores
print scores.mean()

[ 0.67741935  0.70491803  0.73770492  0.74576271  0.70175439]
0.713511880698


In [78]:
print cvec.vocabulary_.get('retry')
cvec.vocabulary_.get('set to fixed')

4


17

In [98]:
idx = pd.Series(cvec.vocabulary_)
b = idx.order().index
feat_imp = pd.DataFrame(classifier.coef_,columns=b.values)
feat_imp
feat_imp.rename(columns=lambda x: x.replace(" ","_"),inplace=True)
feat_imp
#feat_imp.abs().max(axis=1)

,duplicate,change_request,bucket,fix_description,retry,back,dev,in_progress,duplicate_of,requirement,new_requirement,new_requirements,tester_confused,same_as,monitoring,rforct,port_to_tb,set_to_fixed,in_a_while,have_not_seen,fix_desc,requirements,could_not_reproduce,intermittently,intermittent,script_was_incorrect,ready_for_client_testing
0,-3.001696,-2.406517,-4.008630,-2.981628,-4.253231,-1.248572,-1.894315,-3.909329,-3.435988,-4.543430,-4.543430,-4.543430,-4.543430,-3.157135,-3.588775,-2.851351,-4.543430,-4.543430,-4.543430,-4.543430,-4.543430,-4.543430,-4.543430,-4.543430,-4.543430,-4.543430,-4.543430
1,-1.947554,-3.602659,-3.602659,-3.602659,-3.602659,-3.602659,-3.285849,-3.602659,-1.975517,-3.602659,-3.602659,-3.602659,-3.602659,-2.909512,-3.602659,-3.602659,-3.602659,-3.602659,-3.602659,-3.602659,-3.602659,-3.602659,-3.602659,-3.602659,-3.602659,-3.602659,-3.602659
2,-3.252943,-3.773557,-2.223849,-3.773557,-3.773557,-2.113588,-3.773557,-2.416350,-3.225226,-3.773557,-3.773557,-3.773557,-3.773557,-3.080409,-2.322730,-3.773557,-3.773557,-3.773557,-3.773557,-3.773557,-3.773557,-3.773557,-3.773557,-3.773557,-3.773557,-3.773557,-3.773557
3,-3.881256,-1.714855,-3.881256,-3.436238,-3.881256,-2.085163,-2.843250,-3.881256,-3.881256,-3.881256,-3.881256,-3.307860,-3.881256,-3.553343,-3.881256,-2.990038,-3.881256,-3.881256,-3.881256,-3.881256,-3.881256,-2.323606,-3.881256,-3.881256,-3.881256,-3.881256,-3.881256
4,-2.962537,-3.056584,-3.655684,-3.655684,-3.655684,-1.582970,-2.962537,-3.655684,-3.655684,-3.655684,-3.655684,-3.655684,-2.962537,-3.655684,-3.655684,-3.655684,-3.655684,-3.655684,-3.655684,-3.655684,-3.655684,-3.655684,-3.655684,-2.999643,-3.655684,-3.655684,-3.655684


In [76]:
print classifier.coef_

[[-3.0016962  -2.40651735 -4.00862959 -2.98162825 -4.25323087 -1.248572
  -1.89431456 -3.90932905 -3.43598778 -4.54342959 -4.54342959 -4.54342959
  -4.54342959 -3.15713523 -3.58877498 -2.85135058 -4.54342959 -4.54342959
  -4.54342959 -4.54342959 -4.54342959 -4.54342959 -4.54342959 -4.54342959
  -4.54342959 -4.54342959 -4.54342959]
 [-1.94755394 -3.60265898 -3.60265898 -3.60265898 -3.60265898 -3.60265898
  -3.28584926 -3.60265898 -1.97551732 -3.60265898 -3.60265898 -3.60265898
  -3.60265898 -2.9095118  -3.60265898 -3.60265898 -3.60265898 -3.60265898
  -3.60265898 -3.60265898 -3.60265898 -3.60265898 -3.60265898 -3.60265898
  -3.60265898 -3.60265898 -3.60265898]
 [-3.2529427  -3.77355655 -2.22384916 -3.77355655 -3.77355655 -2.11358821
  -3.77355655 -2.41634976 -3.22522602 -3.77355655 -3.77355655 -3.77355655
  -3.77355655 -3.08040937 -2.32273016 -3.77355655 -3.77355655 -3.77355655
  -3.77355655 -3.77355655 -3.77355655 -3.77355655 -3.77355655 -3.77355655
  -3.77355655 -3.77355655 -3.7735565

In [66]:
classifier.classes_

array(['Code defect', 'Duplicate', 'Not reproducible', 'Requirement issue',
       'Tester error'], 
      dtype='|S17')

In [158]:
classifier.classes_

array(['Code defect', 'Duplicate', 'Not reproducible', 'Requirement issue',
       'Tester error'], 
      dtype='|S17')

In [77]:
print label_test.value_counts()
print 'term freq accuracy', accuracy_score(label_test, test_predictions)
cats = ['Code defect', 'Duplicate', 'Not reproducible', 'Requirement issue',
       'Tester error']
tf_df = confusion_matrix(label_test, test_predictions, labels=['Code defect', 'Duplicate', 'Not reproducible', 'Requirement issue',
       'Tester error'])
tf_df = pd.DataFrame(tf_df,index=cats, columns=cats)
tf_df.index.name = 'Actual'
tf_df.columns.name = 'Predicted'
tf_df

Code defect          42
Requirement issue     6
Tester error          6
Not reproducible      5
Duplicate             2
dtype: int64
term freq accuracy 0.770491803279


Predicted,Code defect,Duplicate,Not reproducible,Requirement issue,Tester error
Actual,,,,,
Code defect,37,0,1,2,2
Duplicate,0,2,0,0,0
Not reproducible,2,0,3,0,0
Requirement issue,3,0,0,3,0
Tester error,4,0,0,0,2


In [78]:
print idf_label_test.value_counts()
print 'idf accuracy', accuracy_score(idf_label_test, idf_test_predictions)
idf_df = confusion_matrix(idf_label_test, idf_test_predictions, labels=['Code defect', 'Duplicate', 'Not reproducible', 'Requirement issue',
       'Tester error'])
idf_df = pd.DataFrame(idf_df,index=cats, columns=cats)
idf_df.index.name = 'Actual'
idf_df.columns.name = 'Predicted'
idf_df


Code defect          42
Requirement issue     6
Tester error          6
Not reproducible      5
Duplicate             2
dtype: int64
idf accuracy 0.770491803279


Predicted,Code defect,Duplicate,Not reproducible,Requirement issue,Tester error
Actual,,,,,
Code defect,39,0,1,2,0
Duplicate,0,2,0,0,0
Not reproducible,2,0,3,0,0
Requirement issue,3,0,0,3,0
Tester error,5,1,0,0,0


In [79]:
tf_df

Predicted,Code defect,Duplicate,Not reproducible,Requirement issue,Tester error
Actual,,,,,
Code defect,37,0,1,2,2
Duplicate,0,2,0,0,0
Not reproducible,2,0,3,0,0
Requirement issue,3,0,0,3,0
Tester error,4,0,0,0,2


In [124]:
#pre-cleaned data:
pd.crosstab(label_test, test_predictions)

col_0,Code defect,Duplicate,Not reproducible,Requirement issue,Tester error
Root_Cause,,,,,
Code defect,37,0,1,2,2
Duplicate,1,1,0,0,0
Not reproducible,3,0,2,0,0
Requirement issue,3,0,0,1,2
Tester error,4,0,0,0,2


In [125]:
#pre-cleaned data:
pd.crosstab(idf_label_test, idf_test_predictions)

col_0,Code defect,Duplicate,Not reproducible,Requirement issue
Root_Cause,,,,
Code defect,38,0,1,3
Duplicate,0,2,0,0
Not reproducible,3,0,2,0
Requirement issue,5,0,0,1
Tester error,6,0,0,0


In [126]:
#pre-cleaned data:
print idf_label_test.value_counts()
print 'idf accuracy', accuracy_score(idf_label_test, idf_test_predictions)
idf_df = confusion_matrix(idf_label_test, idf_test_predictions, labels=['Code defect', 'Duplicate', 'Not reproducible', 'Requirement issue',
       'Tester error'])
idf_df = pd.DataFrame(idf_df,index=cats, columns=cats)
idf_df.index.name = 'Actual'
idf_df.columns.name = 'Predicted'
idf_df


Code defect          42
Requirement issue     6
Tester error          6
Not reproducible      5
Duplicate             2
dtype: int64
idf accuracy 0.704918032787


Predicted,Code defect,Duplicate,Not reproducible,Requirement issue,Tester error
Actual,,,,,
Code defect,38,0,1,3,0
Duplicate,0,2,0,0,0
Not reproducible,3,0,2,0,0
Requirement issue,5,0,0,1,0
Tester error,6,0,0,0,0


In [62]:
#stratifiedKFold, cleaned data:
pd.crosstab(idf_label_test, idf_test_predictions,margins=True)

col_0,Code defect,Duplicate,Not reproducible,Requirement issue,Tester error,All
Root_Cause,,,,,,
Code defect,38,2,0,1,1,42
Duplicate,0,1,0,0,0,1
Not reproducible,1,0,3,0,0,4
Requirement issue,2,0,0,3,0,5
Tester error,5,0,0,0,0,5
All,46,3,3,4,1,57


In [61]:
label_test[10:20]

275     Code defect
46      Code defect
284     Code defect
257     Code defect
280    Tester error
66      Code defect
189     Code defect
208     Code defect
37      Code defect
177    Tester error
Name: Root_Cause, dtype: object

In [86]:
raw_data.Defect_ID.loc[167]

226

In [92]:
t = zip(label_test,test_predictions)
t

[('Requirement issue', 'Requirement issue'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Tester error'),
 ('Not reproducible', 'Code defect'),
 ('Requirement issue', 'Code defect'),
 ('Duplicate', 'Duplicate'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Duplicate', 'Duplicate'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Not reproducible', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Not reproducible', 'Not reproducible'),
 ('Requirement issue', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Tester error', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Requirement issue'),
 ('Requirement issue', 'Requirement issue'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Co

In [90]:
v = zip(idf_label_test,idf_test_predictions)
v

[('Requirement issue', 'Requirement issue'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Tester error'),
 ('Not reproducible', 'Code defect'),
 ('Requirement issue', 'Code defect'),
 ('Duplicate', 'Duplicate'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Duplicate', 'Duplicate'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Not reproducible', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Not reproducible', 'Not reproducible'),
 ('Requirement issue', 'Tester error'),
 ('Code defect', 'Code defect'),
 ('Tester error', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Requirement issue'),
 ('Requirement issue', 'Requirement issue'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'Code defect'),
 ('Code defect', 'C

In [155]:
#not sure what this does yet
print classification_report(label_test, test_predictions)

             precision    recall  f1-score   support

Code defect       0.77      0.95      0.85        42
  Duplicate       1.00      1.00      1.00         2
Not reproducible       1.00      0.20      0.33         5
Requirement issue       0.50      0.17      0.25         6
Tester error       0.75      0.50      0.60         6

avg / total       0.77      0.77      0.73        61

